Data Project Rubric: 
Data Analysis with Python (Pandas & Matplotlib)

1. Project Overview 

○   Analysis of reviews of BG3 as of December 14, 2023 (after the 6th patch release), searching for a rating score of 1-10/10 reccommendations to determine if purchasing and playing to game is worth spending the money and time envoled to complete the game. I also did an Analysis of method of access from the reviews (i.e. whether the reviewer purchased the game via Steam or received the game for free). 


○   This data source comes from https://www.kaggle.com/datasets/harisyafie/baldurs-gate-3-steam-reviews/data 309K reviews from Steam digital distibution service, data is dated up to December 14 2023. 


In [96]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import nltk
# nltk.download('stopwords')
import string


2. Data Collection and Loading 

●	Load Data: I used Pandas to load the dataset via a CSV file. 

●	Initial Check: Please see below for the initial disply of the dataset. Then the first few rows and basic information about the dataset, noting column names, types, and missing values.

●	Selection Options:

○	I dropped several columns that were not needed for the analysis (language, voted_up, votes_up,written_during_early_access, votes_funny, weighted_vote_score, comment_count). 

○	The columns that I focused were Reviews, TimeStapm created, Timestamp updated, steam purchase, and recieved for free. 


In [ ]:
BG3 = pd.read_csv("BG3_reviews_updated.csv")

BG3

In [ ]:

BG3.head()


In [ ]:
BG3.shape

rows, columns = BG3.shape
print(f"BG3 dataset has {rows} rows and {columns} columns.")

In [ ]:
BG3.describe()


In [ ]:
BG3 = BG3.drop(['language','voted_up','votes_up','written_during_early_access','votes_funny','weighted_vote_score','comment_count'], axis=1)
BG3

In [ ]:
BG3.info()

3. Data Cleaning and Preparation 

●	Handle Missing Values: I ran for all null values, found that they were only located in the 'Review' column. I left them as is because I extracted the number of reviews that contained a rating out of 1 to 10/10 to use for the analysis of recommendations. I also left them as they would screw the numbers for the method of access that I graphed at the end of the analysis. This resulted in a dictionary with the number of ratings for each 1 to 10/10. 

○	Explain why you did this drop, fill, etc.: I left the data as is, as I am looking for the score out of 10 that the reviewer left those without a review or a score are not counted in the analysis. I am also looking to compare the number of free versions vs purchases that were reviewed. 

●	Data Type Adjustments: I converted the timestamp columns to current date/time format. It was in Linux time (seconds lapsed from January 1, 1970 at 00:00 hrs).

●	Feature Engineering: Created a new column by turning the boolean from 'steam_purchase' and 'received_for_free' to a string of steam or free, then combined the columns to one to get a single graphical representation of the method of access from the dataset. I had to rename the rows that either combined to SteamFree or were null, to both or no response. This can be seen in the below graph. 

○


In [ ]:
BG3.isnull()

BG3.any()

In [ ]:
missing_values = BG3.isna()
print(missing_values)

In [ ]:
missing_count = BG3.isna().sum()
print(missing_count)

In [ ]:
#BG3['timestamp_created'] = pd.to_datetime(BG3['timestamp_created'])
BG3['timestamp_created'] = pd.to_datetime(BG3['timestamp_created'], unit='s')
BG3
#convert from unix 
#unix_timestamp = 1643723400  # example Unix timestamp

#date_object = pd.to_datetime(unix_timestamp, unit='s')
#print(date_object)  # Output: 2022-02-01 00:00:00

In [ ]:

BG3['timestamp_updated'] = pd.to_datetime(BG3['timestamp_updated'], unit='s')
BG3

In [ ]:
column_list = BG3.columns
list(column_list)


In [ ]:
# change booleann to str, True to Steam in column 'steam_purchase'
BG3['steam_purchase'] = BG3['steam_purchase'].replace({True:'True', False:'False'})
BG3.info()

In [ ]:
#change boolean to str in 'recevied_for_free'
BG3['received_for_free'] = BG3['received_for_free'].replace({True: 'True', False:'False'})
BG3.info()

In [ ]:
# change yes in steam column to Steam, change yes in free column to free
# BG3['steam_purchase'].replace('True', 'Steam', inplace=True)
# BG3.replace({'steam_purchase':'True','Steam':}, inplace=True)
BG3['steam_purchase'] = BG3['steam_purchase'].replace({'True':'Steam','False':''})
BG3['received_for_free'] = BG3['received_for_free'].replace({'True':'Free','False':''})
BG3
# build a column to consolidate Free vs Steam purchase
# BG3['steam_purchase'].value_counts()
BG3['received_for_free'].value_counts()

In [ ]:
#concatenate 'steam_purchase' and 'received_for_free' to build new column 'Access_Method'
#df['new column'] = df['column1'] + df['column2']
BG3['Access_Method'] = BG3['steam_purchase'] + BG3['received_for_free']
BG3


In [ ]:
#change 'SteamFree' to 'Both
BG3['Access_Method'] = BG3['Access_Method'].replace({'SteamFree':'Both','':'No Response'})
BG3

In [ ]:
BG3['Access_Method'].value_counts()

In [ ]:
#drop 'steam_purchse' & 'received_for_free' columns
BG3 = BG3.drop(['steam_purchase','received_for_free'],axis=1)
BG3

In [ ]:
# BG3["review"].value_counts()
# count most common words used in 'review' column
# # Use Counter on the 'review' column

most_used_words = Counter(BG3['review'])
print(most_used_words)

In [ ]:
most_used_words

In [ ]:
#Find 10 most common words/phrases in 'reveiw' column

top_N = 10

# Read the CSV file
BG3 = pd.read_csv(r'BG3_reviews_updated.csv', usecols=['review'])

# Get the stopwords
stopwords = nltk.corpus.stopwords.words('english')

# RegEx for stopwords
RE_stopwords = r'\b(?:{})\b'.format('|'.join(stopwords))

# Words to remove (in addition to stopwords and punctuation)
words_to_remove = ['game', 'games', 'play', 'plays','played','3']

# Preprocess text: Convert to lowercase, replace unwanted characters, remove stopwords, punctuation, and specific words
words = (BG3['review']
           .str.lower()  # Convert to lowercase
           .replace([r'\|', RE_stopwords], [' ', ''], regex=True)  # Remove unwanted characters and stopwords
           .str.replace(f'[{string.punctuation}]', '', regex=True)  # Remove punctuation marks
           .str.cat(sep=' ')  # Concatenate all reviews into one large string
           .split()  # Split into individual words
)

# Remove specific words from the list
words = [word for word in words if word not in words_to_remove]

# Generate frequency count using Counter
rslt = pd.DataFrame(Counter(words).most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')

print(rslt)



In [ ]:
def count_and_sum_patterns(BG3, column):
    pattern_counts = {}
    for i in range(1, 11):
        pattern = f"{i}/10"
        count = BG3['review'].str.contains(pattern).sum()
        pattern_counts[pattern] = count
    return pattern_counts

# Count and sum patterns
result = count_and_sum_patterns(BG3, 'review')
print(result)



In [ ]:
# if statement if reveiw column does not contain result .drop(axis=0) 
result 

In [ ]:
total_sum = sum(result.values())

#calculate the percentage of each item
percentages = {key: (value/total_sum) * 100 for key, value in result.items()}

#format the percentages to 2 decimal places
formatted_percentages = {key: round(value, 2) for key, value in percentages.items()}

print("Total sum:", total_sum)


In [ ]:
print("Percentages:", formatted_percentages)

4. Exploratory Data Analysis (EDA) 

●	Descriptive Statistics: I used the dictionary of ratings to get the total number of reviews that contained a rating, then factored the percentages for each score given from the total rated reviews. 

●	Data Visualizations: I graphed the percentages of the ratings given in a bar graph below. I also graphed out the methods of access as described by the dataset in a bar graph. I also searched for the 10 most common words found in the written reviews to gauage where the highlights are for players. 

○	


In [ ]:
summary = BG3.describe()
print(summary)

In [ ]:
summary = BG3.describe()
pd.set_option('display.float_format', '{:.2f}'.format)
print(summary)

In [ ]:
# Plot the result as a bar chart
rslt.plot.bar(rot=0, figsize=(16,10), width=0.8)

In [ ]:
#define ticks and adjust x,y axix to clean up the plot

plt.bar(formatted_percentages.keys(), formatted_percentages.values())
#plt.yticks([0, 100, 10]))
plt.xlabel('Reccommendation out of 10')
plt.ylabel('Reccommendation Percentages')
plt.title("Baldur's Gate 3 Reccomemdations out of 10")
# plt.yticks(np.arange(0, 10, 100))
plt.show()

In [ ]:
BG3['Access_Method'].value_counts().plot(kind='bar', rot=90)

5. Analysis and Insights

●	Findings: Reviewers recommended Baldur's Gate 3 seventy-four (74) percent of the time at a 10/10. There were some outliers for lower scoring with 1/10, 9/10, and 5/10 being the next highest recommendation scores. 

●	Supporting Data: Extraction of rating scores from the available dataset. 
Count_and_sum_patterns: {'1/10': 1799, '2/10': 354, '3/10': 122, '4/10': 81, '5/10': 1002, '6/10': 166, '7/10': 354, '8/10': 676, '9/10': 1574, '10/10': 17830}  
Percentages: {'1/10': 7.51, '2/10': 1.48, '3/10': 0.51, '4/10': 0.34, '5/10': 4.18, '6/10': 0.69, '7/10': 1.48, '8/10': 2.82, '9/10': 6.57, '10/10': 74.42}

●	The majority of reviews from this dataset came from Steam purchases, and the majortiy of ratings are reported as 10/10, therefore, you can surmise that the purchase price and the time needed to complete a run through would be worth both investments. 

6. Conclusion and Recommendations (10 points)

●	Summarize: The majority of reviews came from purchases of the game Baldur's Gate 3; 3079 were free, 278256 were Steam purchases. Of the reviews submitted, 23958 contained a reccommendation rating out of 10. Of those, 74% of players reccommend BG3 at a 10/10. Larian Studios has produced a highly reccommended game that is worth the monetary cost and the time investment to complete a run through of the game.

●	Recommendations: Compile updated data from December 14, 2023 to present to determine the continued popularity of the game and the statistical reccommendations since this data was compiled. Analysis of continued reccommendations will give Larian Studios addtional areas to concentrate on for updates and continued player satisfaction. 


Optional Advanced Section (Bonus)

●	Advanced EDA: Use pair plots or correlation matrices to explore relationships.

●	Dashboards: Create a simple dashboard with Matplotlib or extend to Dash/Streamlit.
